In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_squared_error

/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
# Keras is a deep learning library that wraps the efficient numerical libraries Theano and TensorFlow.
# It provides a clean and simple API that allows you to define and evaluate deep learning models in just a few lines of code.from keras.models import Sequential, load_model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
from keras.layers.advanced_activations import PReLU
from keras.regularizers import l2
import h5py

Using TensorFlow backend.


In [3]:
# define path to save model
import os
model_path = "../../data/Mercedes_Benz_Greener_Manufacturing/model/model_nn.h5"

## 6. Model

In [4]:
# data
dt_all = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/data/dt_all.csv")
dt_train_raw = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/raw/train.csv")
dt_test_raw = pd.read_csv("/media/noahhhhhh/dataScience/proj/competition/data/Mercedes_Benz_Greener_Manufacturing/raw/test.csv")

In [6]:
cols_cat = dt_all.drop("ID", axis = 1).select_dtypes(include = ["object"]).columns.values.tolist()
cols_bin = dt_all.filter(regex = "^X").columns.values.tolist()
cols_bin = [x for x in cols_bin if x not in cols_cat]


In [7]:
dt_test = dt_all[~dt_all["y"].notnull()]
dt_train = dt_all[dt_all["y"].notnull()]
dt_train_dups = dt_train[dt_train[cols_cat + cols_bin].duplicated(keep = False) == True]
dt_train_nonDups = dt_train[dt_train[cols_cat + cols_bin].duplicated(keep = False) == False]

In [45]:
dt_dups_agg = pd.DataFrame({"y" : dt_train_dups.groupby(cols_cat + cols_bin)["y"].apply(lambda x: sorted(x)[math.ceil(len(x) / 2)])}).reset_index()

In [51]:
dt_dups_agg = pd.DataFrame({"y" : dt_train_dups.groupby(cols_cat + cols_bin)["y"].apply(lambda x: sorted(x)[math.ceil(len(x) / 2)])}).reset_index()
dt_nonDups = dt_train_dups[dt_train_dups["y"].isin(dt_dups_agg["y"])]
dt_nonDup_agg = pd.DataFrame({"ID" : dt_nonDups.groupby(cols_cat + cols_bin)["ID"].max()}).reset_index()
dt_nonDups_train = dt_train_dups[dt_train_dups["ID"].isin(dt_nonDup_agg["ID"])]

In [33]:
import math
x = np.array([1, 3, 2, 4])
sorted(x)[math.ceil(len(x) / 2)]

3

In [34]:
def median_noah(series):
    if len(series) % 2 == 0:
        return(sorted(series)[math.ceil(len(series) / 2)])
    else:
        return(np.median(series))
        

In [38]:
median_noah([10, 9, 12, 11])

11.0

In [52]:
def dups(method, dt_all, cols_cat, cols_bin):

   
    dt_test = dt_all[~dt_all["y"].notnull()]
    dt_train = dt_all[dt_all["y"].notnull()]
    dt_train_dups = dt_train[dt_train[cols_cat + cols_bin].duplicated(keep = False) == True]
    dt_train_nonDups = dt_train[dt_train[cols_cat + cols_bin].duplicated(keep = False) == False]

    if method == "max":
        dt_dups_agg = pd.DataFrame({"y" : dt_train_dups.groupby(cols_cat + cols_bin)["y"].max()}).reset_index()
        dt_nonDups = dt_train_dups[dt_train_dups["y"].isin(dt_dups_agg["y"])]
        dt_nonDup_agg = pd.DataFrame({"ID" : dt_nonDups.groupby(cols_cat + cols_bin)["ID"].max()}).reset_index()
        dt_nonDups_train = dt_train_dups[dt_train_dups["ID"].isin(dt_nonDup_agg["ID"])]
    elif method == "min":
        dt_dups_agg = pd.DataFrame({"y" : dt_train_dups.groupby(cols_cat + cols_bin)["y"].min()}).reset_index()
        dt_nonDups = dt_train_dups[dt_train_dups["y"].isin(dt_dups_agg["y"])]
        dt_nonDup_agg = pd.DataFrame({"ID" : dt_nonDups.groupby(cols_cat + cols_bin)["ID"].max()}).reset_index()
        dt_nonDups_train = dt_train_dups[dt_train_dups["ID"].isin(dt_nonDup_agg["ID"])]
    elif method == "median":
        dt_dups_agg = pd.DataFrame({"y" : dt_train_dups.groupby(cols_cat + cols_bin)["y"].apply(lambda x: sorted(x)[math.ceil(len(x) / 2)])}).reset_index()
        dt_nonDups = dt_train_dups[dt_train_dups["y"].isin(dt_dups_agg["y"])]
        dt_nonDup_agg = pd.DataFrame({"ID" : dt_nonDups.groupby(cols_cat + cols_bin)["ID"].max()}).reset_index()
        dt_nonDups_train = dt_train_dups[dt_train_dups["ID"].isin(dt_nonDup_agg["ID"])]
    
    return(pd.concat([dt_nonDups_train, dt_train_nonDups, dt_test]))

In [53]:
dups("median", dt_all, cols_cat, cols_bin)

ID  X0  X1  X10  X100  X101  X102  X103  X104  X105     ...       \
16     553   z  aa    0     1     1     0     1     0     0     ...        
26    6921   w   v    0     0     0     1     0     0     0     ...        
30    4560   y  aa    0     1     1     0     1     0     0     ...        
91    4294   x   b    0     1     1     0     1     0     0     ...        
94    8137  ay   i    0     1     1     0     1     0     0     ...        
103   4848   z   b    0     1     1     0     1     0     0     ...        
109   4879   z   b    0     1     1     0     1     0     0     ...        
135   7691   j   i    0     1     1     0     1     0     0     ...        
198    202   f   c    0     1     1     0     1     0     0     ...        
216   2191   o   l    0     1     1     0     1     0     0     ...        
223   2768   o   l    0     1     1     0     1     0     0     ...        
232   6032   y   a    0     0     1     0     0     0     0     ...        
271   5028  ay   i    0     0     1     0     0     0     0     ...        
273   4426  ay   i    0     0     1     0     0     0     0     ...        
292   5198   t   r    0     1     0     0     0     0     0     ...        
313   4547   f   l    0     1     1     0     1     0     0     ...        
335    847   x  aa    0     1     1     0     1     0     0     ...        
354   8235   j   r    0     0     0     1     0     0     0     ...        
418   2147   h   a    0     0     1     0     1     0     0     ...        
440   1180   z  aa    0     1     1     0     1     0     0     ...        
448   7596   w   v    0     0     0     1     0     0     0     ...        
454   7499  ay   r    0     1     1     0     1     0     0     ...        
458   1067  ay   i    0     0     1     0     0     0     0     ...        
483    722   j  aa    0     0     0     0     0     0     0     ...        
514   8312  ay   i    0     1     1     0     1     0     0     ...        
519    166   x   b    0     1     1     0     1     0     0     ...        
546   1181   t   b    0     0     1     0     0     0     0     ...        
568   6667   j   i    0     1     1     0     1     0     0     ...        
594   8183   y   r    0     0     0     0     0     0     0     ...        
614   5952  ak   v    0     1     1     0     1     0     0     ...        
...    ...  ..  ..  ...   ...   ...   ...   ...   ...   ...     ...        
8388  2358   n   a    0     0     1     0     1     0     0     ...        
8389   338   o   k    0     1     1     0     1     0     0     ...        
8390  7717  ak   s    0     1     1     0     1     0     0     ...        
8391  3526   x   b    0     1     1     0     1     0     0     ...        
8392  3917  ak   l    0     1     1     0     1     0     0     ...        
8393  3969   w   s    0     1     1     0     1     0     0     ...        
8394  7570   o   v    0     1     1     0     1     0     0     ...        
8395  6395   z   b    0     0     1     0     1     0     0     ...        
8396  1544  ay   b    0     1     1     0     1     0     0     ...        
8397  4960   b   b    0     1     1     0     1     0     0     ...        
8398  5389   w   s    0     1     1     0     1     0     0     ...        
8399  1116   n   a    0     0     1     0     1     0     0     ...        
8400  3751   n   a    0     0     1     0     1     0     0     ...        
8401  4662  al  aa    0     1     1     0     1     0     0     ...        
8402    69   z   r    0     0     0     0     0     0     0     ...        
8403  5878   n   a    0     0     1     0     1     0     0     ...        
8404  5667   z  aa    0     1     1     0     1     0     0     ...        
8405  7579   y   a    0     0     1     0     1     0     0     ...        
8406  3002   x  aa    0     0     0     0     0     0     0     ...        
8407  6429   y   r    0     1     1     0     1     0     0     ...        
8408  8246   w   w    0     1     1     0     1     0   

In [5]:
# ids
ids_train = dt_train_raw.ID.values
ids_test = dt_test_raw.ID.values

In [6]:
# r_2 for nn
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [7]:
# X, y, ID
X_train_all = dt_all[dt_all["ID"].isin(ids_train)].as_matrix()
X_test = dt_all[dt_all["ID"].isin(ids_test)].as_matrix()
y_train_all = dt_all[dt_all["ID"].isin(ids_train)].y.values

In [8]:
# keras
def model(dropout_level = 0.25, activation = 'tanh'):
    model = Sequential()
    model.add(Dense(1024, input_dim=input_dims, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(1024, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(1024, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(1024, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(1024, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(768, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(768, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(768, kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))

    model.add(Dense(768, kernel_initializer="he_normal", kernel_regularizer = l2(1.e-5)))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_level))
    
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss = "mean_squared_error", # one may use 'mean_absolute_error' as alternative
                  optimizer = "adam",
                  metrics = [r2_keras, "accuracy"] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [9]:
input_dims = X_train_all.shape[1]
estimator = KerasRegressor(
    build_fn = model, 
    nb_epoch = 300, 
    batch_size = 35,
    verbose = 1
)

In [10]:
# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor = 'val_loss', 
        patience = 20,
        verbose = 1),
    ModelCheckpoint(
        model_path, 
        monitor = 'val_loss', 
        save_best_only = True, 
        verbose = 0)
]

In [11]:
#K-FOLD
n_splits = 5
kf = KFold(n_splits = n_splits)
kf.get_n_splits(X_train_all)

predictions = np.zeros((X_train_all.shape[0], n_splits))
score = 0

for fold, (ind_train, ind_valid) in enumerate(kf.split(X_train_all)):

    X_train, X_valid = X_train_all[ind_train, :], X_train_all[ind_valid, :]
    y_train, y_valid = y_train_all[ind_train], y_train_all[ind_valid]

    # fit estimator
    history = estimator.fit(
        X_train, 
        y_train, 
        batch_size = 64,
        epochs = 500,
        validation_data = (X_valid, y_valid),
        verbose = 2,
        callbacks = callbacks,
        shuffle = True
    )
    
    if os.path.isfile(model_path):
        history.model.load_weights(model_path)
    
    pred = history.model.predict(X_valid)
    
    score_fold = r2_score(y_valid, pred)
    score += score_fold

    print('Fold %d: Score %f'%(fold, score_fold))


score /= n_splits

print('=====================')

print( 'Final Score %f'%score)

print('=====================')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              681984    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
__________

InternalError: Dst tensor is not initialized.
	 [[Node: Const_32 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1024,1024] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Const_32', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).sta